### See https://huggingface.co/blog/fine-tune-xlsr-wav2vec2 for more details

### All required installations

In [1]:
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 56 kB 4.9 MB/s  eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.1.0
    Uninstalling huggingface-hub-0.1.0:
      Successfully uninstalled huggingface-hub-0.1.0
     |████████████████████████████████| 59 kB 9.1 MB/s  eta 0:00:01


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.13.3 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingface-hub 0.1.0 which is incompatible.


In [2]:
! git config --global credential.helper store

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.


### Pick a language to fine-tune XLS-R on

In [41]:
from datasets import load_dataset, load_metric, Audio

#TODO: change language to fr
common_voice_train = load_dataset("common_voice", "fr", split='train[:5000]')
common_voice_test = load_dataset("common_voice", "fr", split="test[:1000]")

Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/fr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/fr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [42]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [43]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [44]:
show_random_elements(common_voice_train, num_examples=10)

,path,audio,sentence
0,/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_20632242.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_20632242.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Pas de sphère de magma multicolore.
1,/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_23156953.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_23156953.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Il s’agit de la limite entre les systèmes Trias et Jurassique.
2,/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_19726972.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_19726972.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}",Il fut le premier ministre du Queensland et membre du premier gouvernement fédéral.
3,/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_19084544.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/548154eb27378f7b35bce9e5da6653439db8b376b5ca5194fe2c1eccd50b342c/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_19084544.mp3', 'array': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...], 'sampling_rate': 48000}","Disponible sur les plates-formes de téléchargement, elle se vend à exemplaires."
4,/root/.cache/huggingface/dataset

We can see that the transcriptions contain some special characters, such as ,.?!;:. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. E.g., the letter "s" has a more or less clear sound, whereas the special character "." does not.

Let's simply remove all characters that don't contribute to the meaning of a word and cannot really be represented by an acoustic sound and normalize the text.

In [45]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    #'^'
    remove_char = ['$', '&', '(', ')', '*', '/', '=', '_', '`', '{', '}', '~', '£', '«', '°', '´', '·', 'º', '»', 'ß', 'ã', 'ä', 'å', 'æ', 'ë', 'ï', 'ð', 'ñ', 'õ', 'ö', 'ø', 'ü', 'ý', 'þ', 'ÿ', 'ā', 'ă', 'ą', 'ć', 'ċ', 'č', 'đ', 'ē', 'ė', 'ę', 'ě', 'ğ', 'ĩ', 'ī', 'ı', 'ķ', 'ľ', 'ł', 'ń', 'ņ', 'ō', 'ŏ', 'ő', 'œ', 'ř', 'ś', 'ş', 'š', 'ū', 'ů', 'ź', 'ż', 'ž', 'ơ', 'ư', 'ǫ', 'ǹ', 'ș', 'ț', 'ɨ', 'ʻ', 'ʾ', 'ʿ', 'ː', '̀', '́', '̂', '̈', '̧', 'α', 'β', 'δ', 'ν', 'π', 'ρ', 'σ', 'к', 'ҫ', 'ጠ', 'ḍ', 'ṅ', 'ṇ', 'ṣ', 'ṭ', 'ṯ', 'ạ', 'ả', 'ầ', 'ậ', 'ắ', 'ề', 'ễ', 'ệ', 'ồ', 'ờ', '–', '—', '’', '…', '€', 'ℵ', '−', 'ぬ', 'の', '§', '®', '±', '½', '×', 'ď', 'ġ', 'ħ', 'ĺ', 'ļ', 'ň', 'ţ', 'ť', 'ũ', 'ų', 'ǀ', 'ǃ', 'ǎ', 'ǔ', 'ə', 'ʉ', 'ʼ', 'ʽ', 'ˢ', '̇', '̐', '̠', '̱', '̲', 'γ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'μ', 'ο', 'ς', 'τ', 'υ', 'ψ', 'ω', 'ό', 'а', 'г', 'е', 'з', 'и', 'м', 'н', 'п', 'р', 'э', 'і', 'ј', 'գ', 'զ', 'ا', 'ب', 'ة', 'د', 'ر', 'ل', 'م', 'ن', 'و', 'ي', 'ወ', 'ደ', 'ẵ', 'ị', 'ộ', 'ợ', 'ủ', 'ử', 'ỳ', '‐', '―', '„', '†', '′', '‹', '›', '₽', 'ℂ', 'ℕ', 'ℝ', 'ℤ', 'ℰ', '→', '∅', '∆', '∈', '∞', '∨', '∼', '≥', '─', '☉', 'い', 'う', 'た', 'つ', 'へ', 'ま', 'め', 'や', 'ゔ', '乃', '京', '北', '扬', '文', '星', '术', '杜', '甌', '美', '西', '貴', '青', '馆', 'ꝑ']
    for i in remove_char:
#         print(i)
        batch["sentence"] = re.sub(f'[{i}]', '', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

"hatted" characters - like â - aren't really used anymore in Turkish and can be replaced by their "un-hatted" equivalent, e.g. a.

Let's write another short mapping function to further simplify the text labels. Remember, the simpler the text labels, the easier it is for the model to learn to predict those labels.

In [46]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[à]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[á]', 'a', batch["sentence"])
    
    batch["sentence"] = re.sub('[è]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[é]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[ê]', 'e', batch["sentence"])
    
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ì]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[í]', 'i', batch["sentence"])
    
    batch["sentence"] = re.sub('[ò]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[ó]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ù]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ú]', 'u', batch["sentence"])
    return batch



In [47]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In CTC, it is common to classify speech chunks into letters, so we will do the same here. Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. It is important to pass the argument batched=True to the map(...) function so that the mapping function has access to all transcriptions at once.

In [48]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [49]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

new_vocab = {'|': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'ç': 27,
 '[UNK]': 28,
 '[PAD]': 29}

In [50]:
bad_characters = [i for i in list(vocab_dict.keys()) if i not in list(new_vocab.keys())]
print(bad_characters)

[]


One should always keep in mind that pre-processing is a very important step before training your model. E.g., we don't want our model to differentiate between a and A just because we forgot to normalize the data. The difference between a and A does not depend on the "sound" of the letter at all, but more on grammatical rules - e.g. use a capitalized letter at the beginning of the sentence.

Cool, now our vocabulary is complete and consists of 32 tokens, which means that the linear layer that we will add on top of the pretrained XLS-R checkpoint will have an output dimension of 32.

### Let's now save the vocabulary as a json file.

In [51]:
# ! brew install git-lfs

In [52]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [53]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [54]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [55]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### In the example above we can see that the audio data is loaded with a sampling rate of 48kHz whereas 16kHz are expected by the model.

In [56]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [57]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

la chaleur est convertie en electricite par des couples thermoelectriques


In [58]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: une paille
Input array shape: (38400,)
Sampling rate: 16000


### Prepare Dataset

In [59]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


In [60]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

### Set-up Trainer

In [61]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [62]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Metrics

In [63]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [64]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [65]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [66]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./',
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=1000,
  gradient_checkpointing=True,
#   fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

comet_ml is installed but `COMET_API_KEY` is not set.


In [69]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [68]:
trainer.train()

NameError: name 'trainer' is not defined

### Evaluation

In [32]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [33]:
common_voice_test_transcription = load_dataset("common_voice", "tr", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration tr-ac779bf2c9f7c09b
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/tr-ac779bf2c9f7c09b/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [34]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())

Prediction:


Reference:
hayatta küçük şeyleri kovalıyor ve yine küçük şeyler için birbirimizi incitiyoruz.
